# KF EM RTS Nomoto

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from vessel_manoeuvring_models.extended_kalman_filter import extended_kalman_filter, simulate, rts_smoother
import vessel_manoeuvring_models.extended_kalman_filter as ekf
from vessel_manoeuvring_models.visualization.plot import plot


import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

In [ ]:
def lambda_f_constructor(K, T_1):
    def lambda_f(x, input):
        delta = input["delta"]
        f = np.array([[x[1], (K * delta - x[1]) / T_1]]).T
        return f

    return lambda_f

In [ ]:
np.random.seed(42)

T_1 = 1.8962353076056344
K = 0.17950970687951323
h = 0.02

lambda_f = lambda_f_constructor(K=K, T_1=T_1)

In [ ]:
def do_simulation(h_, lambda_f, N_=4000):

    t_ = np.arange(0, N_ * h_, h_)

    us = np.deg2rad(
        np.concatenate(
            (
                -10 * np.ones(int(N_ / 4)),
                10 * np.ones(int(N_ / 4)),
                -10 * np.ones(int(N_ / 4)),
                10 * np.ones(int(N_ / 4)),
            )
        )
    )
    data = pd.DataFrame(index=t_)
    data["delta"] = us

    np.random.seed(42)
    E = np.array([[0, 1]]).T
    process_noise = np.deg2rad(0.1)
    ws = np.random.normal(scale=process_noise, size=N_)

    data["psi"] = 0
    data["r"] = 0
    df = simulate(data=data, lambda_f=lambda_f, E=E, ws=ws, state_columns=["psi", "r"])

    measurement_noise = np.deg2rad(1)
    df["epsilon"] = np.random.normal(scale=measurement_noise, size=N_)
    df["psi_measure"] = df["psi"] + df["epsilon"]
    df["psi_deg"] = np.rad2deg(df["psi"])
    df["psi_measure_deg"] = np.rad2deg(df["psi_measure"])
    df["delta_deg"] = np.rad2deg(df["delta"])

    return df

In [ ]:
df = do_simulation(lambda_f=lambda_f, h_=h)
data = pd.DataFrame(index=df.index)
data['psi'] = df['psi_measure']
data['r'] = df['r']
data['delta'] = df['delta']

In [ ]:
plot(dataframes={'sim':data}, keys=["psi",'r'], fig_size=(10,3));

In [ ]:
P_prd = np.diag([np.deg2rad(1), np.deg2rad(0.1)])

Qd = np.deg2rad(np.diag([0.1]))

Rd = np.diag([np.deg2rad(1)])

E = np.array(
    [[0], [1]],
)

C_ = np.array([[1, 0]])

Cd = C_

In [ ]:
def lambda_jacobian_constructor(h, K, T_1):
    def lambda_jacobian(x, input):

        delta = input["delta"]
        r = x[1]

        jac = np.array(
            [
                [1, h],
                [0, 1 - h / T_1],
            ]
        )
        return jac

    return lambda_jacobian

In [ ]:
lambda_jacobian = lambda_jacobian_constructor(h=h, K=K, T_1=T_1)

In [ ]:
state_columns = ['psi','r']
time_steps = extended_kalman_filter(no_states=2, 
                                    no_measurement_states=1, 
                                    P_prd=P_prd, 
                                    lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian, 
                                    data=data, 
                                    Qd=Qd, 
                                    Rd=Rd, 
                                    E=E, 
                                    Cd=Cd, 
                                    state_columns = state_columns,
                                    measurement_columns=['psi'])

In [ ]:
df_kalman = ekf.time_steps_to_df(time_steps, state_columns=state_columns)

In [ ]:
time_steps_smooth = ekf.rts_smoother(
            time_steps=time_steps,
            lambda_jacobian=lambda_jacobian,
            Qd=Qd,
            lambda_f=lambda_f,
            E=E,
        )
df_smooth = ekf.time_steps_to_df(time_steps_smooth, state_columns=state_columns)

In [ ]:
fig_size=(10,3)
plot(dataframes={'real':df, 
                 'measure':data, 
                 #'kalman':df_kalman, 
                 'smooth':df_smooth}, 
                 keys=["psi",'r'], fig_size=fig_size);

## Parameter variation

In [ ]:
def vary(K,T_1):
    
    # Vary:
    lambda_f = lambda_f_constructor(K=K, T_1=T_1)
    lambda_jacobian = lambda_jacobian_constructor(h=h, K=K, T_1=T_1)
    
    # Kalman filter:
    time_steps = extended_kalman_filter(no_states=2, 
                                    no_measurement_states=1, 
                                    P_prd=P_prd, 
                                    lambda_f=lambda_f, 
                                    lambda_jacobian=lambda_jacobian, 
                                    data=data, 
                                    Qd=Qd, 
                                    Rd=Rd, 
                                    E=E, 
                                    Cd=Cd, 
                                    state_columns = state_columns,
                                    measurement_columns=['psi'])
    
    #return time_steps
    
    # RTS smoother:
    time_steps_smooth = ekf.rts_smoother(
            time_steps=time_steps,
            lambda_jacobian=lambda_jacobian,
            Qd=Qd,
            lambda_f=lambda_f,
            E=E,
        )
    
    return time_steps_smooth
    

In [ ]:
T_1s = np.linspace(0.5*T_1, 1.5*T_1, 5)

time_steps_vary = {}
for T_1_ in T_1s:
    time_steps_vary[T_1_] = vary(K=K, T_1=T_1_)

In [ ]:
dataframes_vary = {f'T:{np.round(T_1_,2)}':ekf.time_steps_to_df(time_steps_, state_columns=state_columns) for T_1_,time_steps_ in time_steps_vary.items()}

styles = {f'T:{np.round(T_1_,2)}':{'style':'-', 'alpha':0.5, 'lw':0.5} for T_1_,time_steps_ in time_steps_vary.items()}

dataframes_vary['real'] = df
styles['real'] = {
    'lw':1,
}
plot(dataframes_vary, 
     keys=["psi",'r'], 
     styles=styles,
     fig_size=fig_size);
    
    

In [ ]:
dataframes_vary_variances = {f'T:{np.round(T_1_,2)}':pd.DataFrame(ekf.variance(time_steps_).T, columns=state_columns).iloc[1000:-1000] for T_1_,time_steps_ in time_steps_vary.items()}

fig = plot(dataframes_vary_variances, 
                 keys=["psi",'r'], fig_size=fig_size);

fig.suptitle('Variances');
plt.tight_layout()


In [ ]:
from scipy.stats import multivariate_normal

def _loglikelihood(time_step):
    
    mean = time_step['x_hat'].flatten()
    cov = time_step['P_hat']
    x_prd = time_step['x_prd']
    #cov = time_step['P_prd']
    rv = multivariate_normal(mean=mean, cov=cov)
    return rv.logpdf(x_prd.flatten())
    
    
def loglikelihood(time_steps):
    
    #x_hats = ekf.x_hat(time_steps)
    #x_prds = ekf.x_prd(time_steps)
    #cov = time_steps[0]['P_hat']
    #
    #epsilon = x_prds - x_hats 
    #rv = multivariate_normal(mean=np.zeros(x_hats.shape[0]), cov=cov)
    #loglikelihood = np.sum(rv.logpdf(epsilon.T))    
    
    loglikelihood=0
    for time_step in time_steps:
        loglikelihood+=_loglikelihood(time_step)
        
    
        
    return loglikelihood


In [ ]:
x_hats = ekf.x_hat(time_steps)
np.zeros(x_hats.shape[0])

In [ ]:
loglikelihood(time_steps)

In [ ]:
loglikelihoods = np.zeros(len(time_steps_vary))
for i,(T_1_, time_steps_) in enumerate(time_steps_vary.items()):
    loglikelihoods[i] = loglikelihood(time_steps_)
    

In [ ]:
fig,ax=plt.subplots()
ax.plot(T_1s, loglikelihoods,'.-', label='parameter variation');
ax.plot([T_1,T_1],[np.min(loglikelihoods), np.max(loglikelihoods)],'r--', label=r'real $T_1$')
ax.set_ylabel('likelihood')
ax.set_xlabel(r'$T_1$');
ax.legend();



In [ ]:
from scipy.optimize import minimize

def fun(x):
    T_1_ = x[0]
    K_ = x[1]
    
    time_steps = vary(K=K_, T_1=T_1_)
    return -loglikelihood(time_steps)

In [ ]:
x0=[T_1*1.5, K*1.5]
res = minimize(fun, x0=x0, bounds=[(0.01,np.inf), (0.01,np.inf)], tol=0.001)

In [ ]:
res

In [ ]:
T_1

In [ ]:
K

In [ ]:
time_steps_optimized = vary(K=res.x[1], T_1=res.x[0])
df_sim_identified = simulate(data=data, lambda_f=lambda_f_constructor(K=res.x[1], T_1=res.x[0]), state_columns=["psi", "r"], hidden_state_columns=["r"])

fig_size=(10,3)
plot(dataframes={
                 'measure':data,
                 'real':df,
                 'filter identified': ekf.time_steps_to_df(time_steps_optimized, state_columns=state_columns),
                 'sim identified': df_sim_identified,
                },
                keys=["psi",'r'], fig_size=fig_size);